In [ ]:
# We call the EP method

# 		time <- system.time(ret <- epMPL(problemInfo, lengthScaleItems, lengthScaleUsers))

def epMPL(problemInfo, lengthScaleItems, lengthScaleUsers):

    # We preprocess the problemInfo structure

    problemInfo = preprocessProblemInfo(problemInfo)

    # We select the pseudo inputs for the users and for the items

    problemInfo.indexPseudoInputUsers = sample(1 : length(unique(problemInfo$userIds)), problemInfo$nPseudoInputUsers)
    problemInfo.indexPseudoInputItems = sample(1 : length(unique(problemInfo$ratingIds)), problemInfo$nPseudoInputItems)

    problemInfo.lengthScaleItems = lengthScaleItems
    problemInfo.lengthScaleUsers = lengthScaleUsers

    epMPLinternal <- function(problemInfo) {

	# We add the additional structure for the FITC approximation

	problemInfo <- addFITCauxiliarVariables(problemInfo)

	# Total number of different pairs of items rated and the number of users

	nRatings <- length(unique(problemInfo$ratingIds))
	nUsers <- length(unique(problemInfo$userIds))
	d <- problemInfo$d

	# We initialize the posterior approximation and the factor approximations

	# The first approximate factor

	f1Hat <- list(mu = matrix(0, nUsers, nRatings), vu = matrix(Inf, nUsers, nRatings))

	# The second approximate factor

	f2Hat <- list(mu = matrix(0, nUsers, nRatings), vu = matrix(Inf, nUsers, nRatings),
		mw = matrix(0, nUsers, d), vw = matrix(Inf, nUsers, d),
		mh = matrix(0, d, nRatings), vh = matrix(Inf, d, nRatings))

	# The third approximate factor

	f3Hat <- list(mw = matrix(0, nUsers, d), vw = matrix(Inf, nUsers, d))

	# The fourth approximate factor

	f4Hat <- list(mh = matrix(0, d, nRatings), vh = matrix(Inf, d, nRatings))

	# The posterior approximation

	a <- list(mu = matrix(0, nUsers, nRatings), vu = matrix(Inf, nUsers, nRatings),
		mw = matrix(0, nUsers, d), vw = matrix(1, nUsers, d),
		mh = matrix(0, d, nRatings), vh = matrix(Inf, d, nRatings))

	##
	## We start refinint the factors for the first iteration
	## 

	# We refine the fourth approximate factor

	a$vh <- f4Hat$vh <- matrix(problemInfo$fitcItems$diagKn, d, nRatings, byrow = T)

	# We refine the third approximate factor

	a$vw <- f3Hat$vw <- matrix(problemInfo$fitcUsers$diagKn, nUsers, d)

	# We refine the second approximate factor

	a$vu <- f2Hat$vu <- f3Hat$vw %*% f4Hat$vh

	# The variational solution is initialized to NULL

	variationalSolution <- NULL

	##
	## Main loop of EP
	##

	# We check for an initial solution

	damping <- 0.5
	convergence <- F
	iteration <- 1
	while ((!convergence && iteration <= 1000) || iteration <= 3) {

		aOld <- a

		##
		## We refine the first approximate factor
		##

		mOld <- f2Hat$mu
		vOld <- f2Hat$vu
		Y <- problemInfo$ratingMatrix

		logZ <- pnorm(Y * mOld / sqrt(vOld + 1), log.p = T)
		ratio <- exp(-logZ + dnorm(mOld / sqrt(vOld + 1), log = T))
		alpha <- ratio * Y / sqrt(vOld + 1)
		beta <- -ratio * (Y * mOld / sqrt(vOld + 1) + ratio) / (1 + vOld)
                eta2HatNew <- -beta / (1 + beta * vOld)
                eta1HatNew <- (alpha - mOld * beta) / (1 + beta * vOld)

		vuHatNew <- eta2HatNew^-1
		muHatNew <- eta1HatNew / eta2HatNew

		muHatNew[ is.infinite(vuHatNew) ] <- 0
		vuHatNew[ is.infinite(vuHatNew) ] <- 1e300

		index <- which(vuHatNew < 0)
		muHatNew[ index ] <- f1Hat$mu[ index ]
		vuHatNew[ index ] <- f1Hat$vu[ index ]

			# We do damping

		f1Hat$mu <- damping * muHatNew / vuHatNew + (1 - damping) * f1Hat$mu / f1Hat$vu
		f1Hat$vu <- (damping * vuHatNew^-1 + (1 - damping) * f1Hat$vu^-1)^-1
		f1Hat$mu <- f1Hat$vu * f1Hat$mu

		##
	      	## We refine the second approximate factor
		##

			# We create the rating entries for the variational method

		ratingEntries <- cbind(problemInfo$userIds, problemInfo$ratingIds,
			f1Hat$mu[ cbind(problemInfo$userIds, problemInfo$ratingIds) ],
			f1Hat$vu[ cbind(problemInfo$userIds, problemInfo$ratingIds) ])
		
			# We call the optimization method using the previous solution if iteration > 1

		if (is.null(variationalSolution))
			ret <- bvpcadFast(ratingEntries, d, f3Hat$mw, f3Hat$vw, t(f4Hat$mh), t(f4Hat$vh), 0)
		else
			ret <- bvpcadFast(ratingEntries, d, f3Hat$mw, f3Hat$vw, t(f4Hat$mh), t(f4Hat$vh), 0, variationalSolution)

		variationalSolution <- ret
		vbLowerBound <- ret$bound

			# We update the second approximate factor

		muNew <- ret$mP %*% t(ret$mQ)
		vuNew <- ret$mP^2 %*% t(ret$vQ) + ret$vP %*% t(ret$mQ^2) + ret$vP %*% t(ret$vQ)

		vuHatNew <- 1 / (1 / vuNew - 1 / f1Hat$vu)
		muHatNew <- vuHatNew * (muNew / vuNew - f1Hat$mu / f1Hat$vu)
		vwHatNew <- 1 / (1 / ret$vP - 1 / f3Hat$vw)
		mwHatNew <- vwHatNew * (ret$mP / ret$vP - f3Hat$mw / f3Hat$vw)
		vhHatNew <- 1 / (1 / t(ret$vQ) - 1 / f4Hat$vh)
		mhHatNew <- vhHatNew * (t(ret$mQ / ret$vQ) - f4Hat$mh / f4Hat$vh)

			# We update only those terms with positive variances

		index <- which(vuHatNew < 0)
		vuHatNew[ index ] <- f2Hat$vu[ index ]
		muHatNew[ index ] <- f2Hat$mu[ index ]
		index <- which(vwHatNew < 0)
		vwHatNew[ index ] <- f2Hat$vw[ index ]
		mwHatNew[ index ] <- f2Hat$mw[ index ]
		index <- which(vhHatNew < 0)
		vhHatNew[ index ] <- f2Hat$vh[ index ]
		mhHatNew[ index ] <- f2Hat$mh[ index ]

			# We do damping

		f2Hat$mu <- damping * muHatNew / vuHatNew + (1 - damping) * f2Hat$mu / f2Hat$vu
		f2Hat$vu <- (damping * vuHatNew^-1 + (1 - damping) * f2Hat$vu^-1)^-1
		f2Hat$mu <- f2Hat$vu * f2Hat$mu

		f2Hat$mw <- damping * mwHatNew / vwHatNew + (1 - damping) * f2Hat$mw / f2Hat$vw
		f2Hat$vw <- (damping * vwHatNew^-1 + (1 - damping) * f2Hat$vw^-1)^-1
		f2Hat$mw <- f2Hat$vw * f2Hat$mw

		f2Hat$mh <- damping * mhHatNew / vhHatNew + (1 - damping) * f2Hat$mh / f2Hat$vh
		f2Hat$vh <- (damping * vhHatNew^-1 + (1 - damping) * f2Hat$vh^-1)^-1
		f2Hat$mh <- f2Hat$vh * f2Hat$mh

		##
		## We refine the second approximate factor
		##

		for (i in 1 : d) {

			# We refine the approximate factor for the Gaussian process using the FITC approximation

			ret <- computeTitledDistribution(problemInfo$fitcItems$D, problemInfo$fitcItems$P, problemInfo$fitcItems$R,
				problemInfo$fitcItems$PRt, f2Hat$mh[ i, ], f2Hat$vh[ i, ])
			mhNew <- ret$mNew
			vhNew <- ret$vNew

			# We update the fourth approximate factor
		
			vhHatNew <- 1 / (1 / vhNew - 1 / f2Hat$vh[ i, ])
			mhHatNew <- vhHatNew * (mhNew / vhNew - f2Hat$mh[ i, ] / f2Hat$vh[ i, ])
			
			# We only update those terms with positive variances

			index <- which(vhHatNew < 0)
			vhHatNew[ index ] <- f4Hat$vh[ i, index ]
			mhHatNew[ index ] <- f4Hat$mh[ i, index ]

			# We do damping

			f4Hat$mh[ i, ] <- damping * mhHatNew / vhHatNew + (1 - damping) * f4Hat$mh[ i, ] / f4Hat$vh[ i, ]
			f4Hat$vh[ i, ] <- (damping * vhHatNew^-1 + (1 - damping) * f4Hat$vh[ i, ]^-1)^-1
			f4Hat$mh[ i, ] <- f4Hat$vh[ i, ] * f4Hat$mh[ i, ]
		}

		##
		## We refine the third approximate factor
		##

		for (i in 1 : d) {

			# We refine the approximate factor for the Gaussian process using the FITC approximation

			ret <- computeTitledDistribution(problemInfo$fitcUsers$D, problemInfo$fitcUsers$P, problemInfo$fitcUsers$R,
				problemInfo$fitcUsers$PRt, f2Hat$mw[ , i ], f2Hat$vw[ , i ])
			mwNew <- ret$mNew
			vwNew <- ret$vNew

			# We update the fourth approximate factor
		
			vwHatNew <- 1 / (1 / vwNew - 1 / f2Hat$vw[ , i ])
			mwHatNew <- vwHatNew * (mwNew / vwNew - f2Hat$mw[ , i ] / f2Hat$vw[ , i ])
			
			# We only update those terms with positive variances

			index <- which(vwHatNew < 0)
			vwHatNew[ index ] <- f3Hat$vw[ index, i ]
			mwHatNew[ index ] <- f3Hat$mw[ index, i ]

			# We do damping

			f3Hat$mw[ , i ] <- damping * mwHatNew / vwHatNew + (1 - damping) * f3Hat$mw[ , i ] / f3Hat$vw[ , i ]
			f3Hat$vw[ , i ] <- (damping * vwHatNew^-1 + (1 - damping) * f3Hat$vw[ , i ]^-1)^-1
			f3Hat$mw[ , i ] <- f3Hat$vw[ , i ] * f3Hat$mw[ , i ]
		}

		# We update the posterior approximation

		a$vu <- 1 / (1 / f1Hat$vu + 1 / f2Hat$vu)
		a$mu <- a$vu * (f1Hat$mu / f1Hat$vu + f2Hat$mu / f2Hat$vu)

		a$vw <- 1 / (1 / f3Hat$vw + 1 / f2Hat$vw)
		a$mw <- a$vw * (f3Hat$mw / f3Hat$vw + f2Hat$mw / f2Hat$vw)

		a$vh <- 1 / (1 / f4Hat$vh + 1 / f2Hat$vh)
		a$mh <- a$vh * (f4Hat$mh / f4Hat$vh + f2Hat$mh / f2Hat$vh)

		# We update the damping parameter

		damping <- damping * 0.95

		# We check for convergence

		change <- max(abs(a$mu[ cbind(problemInfo$userIds, problemInfo$ratingIds) ] -
			aOld$mu[ cbind(problemInfo$userIds, problemInfo$ratingIds) ]))
		if (change < 1e-2)
			convergence <- T
		else 
			convergence <- F

		cat(iteration, change, "\n")

		iteration <- iteration + 1
	}

	# We estimate the evidence

	evidence <- computeEvidence(a, problemInfo, f1Hat, f2Hat, f3Hat, f4Hat, vbLowerBound, nRatings, nUsers)

	# We return the posterior approximation

	list(a = a, evidence = evidence, problemInfo = problemInfo,
		f1Hat = f1Hat, f2Hat = f2Hat, f3Hat = f3Hat, f4Hat = f4Hat,
		vbLowerBound = vbLowerBound, variationalSolution = variationalSolution)
}



In [ ]:
# pred <- sign(predictMPL(ret, userIdsTest, itemFeaturesAtest, itemFeaturesBtest)$m)